In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn import model_selection
import numpy as np

In [3]:
df = pd.read_pickle('results.pickle')

In [5]:
df.drop(['タイム', '着差', '着順', '馬名', 'ﾀｲﾑ指数', '通過', '上り', '調教ﾀｲﾑ', '厩舎ｺﾒﾝﾄ', '賞金(万円)'], axis=1, inplace=True)

In [6]:
df.isnull().sum()

枠番         0
馬番         0
性齢         0
斤量         0
騎手         0
単勝         0
人気       166
馬体重        0
備考     34601
調教師        0
馬主         0
dtype: int64

In [7]:
df['性別'] = df['性齢'].str[0]
df['年齢'] = df['性齢'].str[1:3]
df['年齢'] = df['年齢'].astype(int)
df.drop('性齢', axis=1,inplace=True)

In [8]:
df['場所'] =df['調教師'] .str[1:2]
df['調教師'] = df['調教師'].str[4:]

In [9]:
result = []
for i  in df['馬体重'].str[4:].str.replace(')','', regex=True).str[0]:
    if i=='+':
        result.append(1)
    elif i=='-':
        result.append(2)
    else:
        result.append(0)

df['加減'] = result
df['体重増減'] = df['馬体重'].str[4:].str.replace(')','', regex=True).str[1:]
df.loc[(df.体重増減 == ''), '体重増減'] = '0'
df['体重増減'] = df['体重増減'].astype(int)
df['馬体重'] = df['馬体重'].str[0:3]
# df.loc[(df.馬体重 !='計不'), ['馬体重']].astype(float).mean().round().astype(int)
df.loc[(df.馬体重=='計不'), '馬体重'] = '471'
df['馬体重'] = df['馬体重'].astype(int)


In [10]:
result = []

for i in df['備考']:
    if i=='出遅れ':
        result.append(1)
    else:
        result.append(0)

df['出遅れ'] = result
df.drop('備考', axis=1, inplace=True)

In [11]:
df.loc[(df.単勝=='---'), '単勝'] = 0

In [12]:
df['単勝'] = df['単勝'].astype(float).round().astype(int)

In [13]:
from sklearn.ensemble import RandomForestRegressor
dummy = df[['単勝','騎手','場所','性別']]
dummy = pd.get_dummies(dummy)

known_tansyo = dummy[dummy.単勝.notnull()].values  
unknown_tansyo = dummy[dummy.単勝 == 0].values


X = known_tansyo[:, 1:]  
y = known_tansyo[:, 0]

rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
rfr.fit(X, y)

predictedTansyo = rfr.predict(unknown_tansyo[:, 1::])
df.loc[(df.単勝 == 0), '単勝'] = predictedTansyo.round().astype(int)

In [14]:
from sklearn.ensemble import RandomForestRegressor
dummy = df[['人気','騎手','場所','性別']]
dummy = pd.get_dummies(dummy)

known_tansyo = dummy[dummy.人気.notnull()].values  
unknown_tansyo = dummy[dummy.人気.isnull()].values


X = known_tansyo[:, 1:]  
y = known_tansyo[:, 0]

rfr = RandomForestRegressor(random_state=0, n_estimators=100, n_jobs=-1)
rfr.fit(X, y)

predictedTansyo = rfr.predict(unknown_tansyo[:, 1::])
df.loc[(df.人気.isnull()), '人気'] = predictedTansyo

In [30]:
# dummy = df[['騎手','場所','性別','調教師','馬主']]
dummy = df[['場所','性別']]
dummy = pd.get_dummies(dummy)
newDf = pd.concat([df, dummy], axis=1)
newDf.drop(['騎手','場所','性別','調教師','馬主'],axis=1,inplace=True)

In [38]:
newDf

,枠番,馬番,斤量,単勝,人気,馬体重,年齢,加減,体重増減,出遅れ,場所_地,場所_外,場所_東,場所_西,性別_セ,性別_牝,性別_牡
202201010101,1,1,54.0,1,1.0,436,2,0,0,0,0,0,0,1,0,0,1
202201010101,3,3,54.0,11,3.0,450,2,2,6,1,0,0,1,0,0,0,1
202201010101,5,5,54.0,4,2.0,412,2,2,6,0,0,0,1,0,0,0,1
202201010101,2,2,54.0,17,4.0,432,2,2,2,0,0,0,1,0,0,0,1
202201010101,4,4,54.0,28,5.0,460,2,1,6,1,0,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202210040812,6,11,57.0,30,9.0,476,4,2,12,1,0,0,0,1,0,0,1
202210040812,3,5,49.0,139,15.0,444,3,1,6,0,0,0,0,1,0,1,0
202210040812,4,6,52.0,83,13.0,502,3,1,10,1,0,0,0,1,1,0,0
202210040812,7,13,54.0,7,5.0,524,3,1,2,0,0,0,0,1,0,0,1


In [31]:
X = newDf.drop('出遅れ',axis=1)
y = newDf['出遅れ']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=0.2, random_state = 42)
kfold = model_selection.KFold(n_splits = 5) 
scores = {}
lgbm_clf = LGBMClassifier(eval_metric='accuracy')
lgbm_clf.fit(X_train, y_train)
results = model_selection.cross_val_score(lgbm_clf, X_test, y_test, cv = kfold) 
scores[('lightGBM', 'train_score')] = results.mean()
scores[('lightGBM', 'test_score')] = lgbm_clf.score(X_test, y_test)

[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric
[LightGBM] [Warning] Unknown parameter: eval_metric


In [32]:
pd.Series(scores).unstack()

,test_score,train_score
lightGBM,0.767683,0.762811


In [33]:
from sklearn.metrics import accuracy_score 
y_pred = lgbm_clf.predict(X_test)
accuracy_score(y_test,y_pred) 

0.7676831850910631

In [35]:
X_test[['枠番','馬番','斤量','単勝','人気','馬体重','年齢','加減','体重増減']]

,枠番,馬番,斤量,単勝,人気,馬体重,年齢,加減,体重増減
202207010510,2,2,54.0,8,3.0,444,3,2,8
202204020310,7,12,55.0,4,2.0,470,4,1,6
202210040304,5,9,54.0,6,3.0,432,3,1,2
202204010402,7,13,52.0,415,15.0,432,3,1,16
202206050503,1,2,51.0,557,16.0,426,2,1,6
...,...,...,...,...,...,...,...,...,...
202209050108,7,9,57.0,56,8.0,520,4,1,4
202206010202,3,5,54.0,226,15.0,482,3,1,8
202206040710,2,3,57.0,194,16.0,512,8,1,4
202203020711,7,13,53.0,9,4.0,452,5,2,2


In [36]:
proba = lgbm_clf.predict_proba(X_test)

In [20]:
proba[91]

array([0.48523302, 0.51476698])

In [37]:
import pickle
filename = 'model.pkl'
pickle.dump(lgbm_clf,open(filename,'wb'))

In [22]:
num = 0
test = []
for i in y_pred:
    if(i==1):
        test.append(num)
        print(num)
    num = num + 1

68
91
99
151
196
318
397
528
542
562
593
606
610
631
637
668
685
838
873
892
976
1015
1029
1044
1143
1189
1220
1294
1399
1516
1536
1585
1628
1658
1794
1884
1928
1992
2073
2158
2233
2444
2487
2494
2511
2678
2729
3054
3076
3218
3305
3709
3803
3861
3982
4057
4295
4448
4551
4572
4659
4732
4797
4863
4996
5135
5194
5264
5412
5650
5776
5810
5822
5826
5868
5878
5882
5892
5921
5931
5972
6131
6191
6202
6423
6495
6498
6520
6661
6691
6704
6710
6862
6946
7138
7234
7236
7330
7488
7527
7592
7648
7680
7696
7815
7863
7906
7998
8000
8049
8099
8448
8459
8499
8597
8672
8701
8722
8744
8765
8879
8947
9051
9088
9091
9191
9302
9363
9392
9418
9430


In [26]:
X_test[['枠番','馬番','斤量','単勝','人気','馬体重','年齢','加減','体重増減']].iloc[test][:50]

,枠番,馬番,斤量,単勝,人気,馬体重,年齢,加減,体重増減
202206030111,1,1,57.0,17,6.0,464,6,0,0
202205020312,3,4,57.0,7,4.0,422,4,2,10
202207040206,4,8,54.0,352,16.0,404,3,2,18
202204030106,5,10,54.0,325,17.0,410,2,0,0
202207060208,8,18,54.0,344,15.0,440,3,2,8
202203020603,8,15,56.0,156,12.0,580,3,0,0
202205030105,6,8,54.0,147,12.0,470,2,0,0
202209040312,6,11,55.0,30,8.0,442,4,1,6
202207030302,1,2,56.0,389,15.0,488,3,0,0
202210010401,6,10,53.0,92,11.0,406,3,0,0


In [24]:
pd.DataFrame(lgbm_clf.feature_importances_, index=X.columns)[:20]

,0
枠番,86
馬番,144
斤量,157
単勝,266
人気,104
馬体重,350
年齢,117
加減,38
体重増減,126
騎手_イーガン,1
